Scraping McDonalds' Menu

In [5]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import os

# Set output directory
output_dir = r"C:\Users\asmit\OneDrive\Desktop\fast_food_nutrition\data"
os.makedirs(output_dir, exist_ok=True)

# Unique file paths
mcd_source_path = os.path.join(output_dir, 'mcdonalds_page_source.html')
mcd_csv_path = os.path.join(output_dir, 'mcdonalds_nutrition.csv')

# Configure Chrome options
chrome_options = Options()
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--window-size=1920,1080")
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")

service = Service()
driver = webdriver.Chrome(service=service, options=chrome_options)

try:
    url = "https://www.nutritionix.com/mcdonalds/menu/premium"
    driver.get(url)
    time.sleep(3)

    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Save page source
    with open(mcd_source_path, 'w', encoding='utf-8') as f:
        f.write(soup.prettify())
    print(f"Saved page source to: {mcd_source_path}")

    menu_data = []

    # Find nutrition table
    table = soup.find('table', class_='tblCompare')
    if not table:
        raise Exception("Nutrition table not found")

    rows = table.find_all('tr', class_=['odd', 'even'])

    for row in rows:
        name_cell = row.find('td', class_='al')
        if not name_cell:
            continue
        name_link = name_cell.find('a', class_='nmItem')
        item_name = name_link.text.strip() if name_link else "Unknown"

        nutrition_cells = row.find_all('td', class_='col')
        if len(nutrition_cells) < 10:
            continue

        menu_data.append({
            'Item': item_name,
            'Calories': nutrition_cells[0].text.strip(),
            'Total Fat (g)': nutrition_cells[1].text.strip(),
            'Saturated Fat (g)': nutrition_cells[2].text.strip(),
            'Trans Fat (g)': nutrition_cells[3].text.strip(),
            'Cholesterol (mg)': nutrition_cells[4].text.strip(),
            'Sodium (mg)': nutrition_cells[5].text.strip().replace(',', ''),
            'Total Carbohydrates (g)': nutrition_cells[6].text.strip(),
            'Dietary Fiber (g)': nutrition_cells[7].text.strip(),
            'Sugars (g)': nutrition_cells[8].text.strip(),
            'Protein (g)': nutrition_cells[9].text.strip()
        })

    df = pd.DataFrame(menu_data)
    df.to_csv(mcd_csv_path, index=False)
    print(f" Successfully saved {len(df)} items to: {mcd_csv_path}")

except Exception as e:
    print(f"\n Error: {e}")
    screenshot_path = os.path.join(output_dir, 'mcdonalds_error_screenshot.png')
    driver.save_screenshot(screenshot_path)
    print(f"Screenshot saved to: {screenshot_path}")
    print(f"Page source saved to: {mcd_source_path}")

finally:
    driver.quit()

Loading page...
Saved page source to: C:\Users\asmit\OneDrive\Desktop\fast_food_nutrition\data\mcdonalds_page_source.html
✅ Successfully saved 193 items to: C:\Users\asmit\OneDrive\Desktop\fast_food_nutrition\data\mcdonalds_nutrition.csv


Scraping Chick Fil A menu

In [3]:
# Configure Chrome options
chrome_options = Options()
# chrome_options.add_argument("--headless")  # Uncomment when done testing
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--window-size=1920,1080")
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")

service = Service()
driver = webdriver.Chrome(service=service, options=chrome_options)

try:
    cfa_url = "https://www.nutritionix.com/chick-fil-a/menu/premium?desktop"
    driver.get(cfa_url)
    
    time.sleep(3)  # Let JavaScript render

    # Parse the page source
    cfa_soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Save HTML locally
    cfa_source_path = r"C:\Users\asmit\OneDrive\Desktop\fast_food_nutrition\data\cfa_page_source.html"
    with open(cfa_source_path, 'w', encoding='utf-8') as f:
        f.write(cfa_soup.prettify())
    print(f"Saved page source to: {cfa_source_path}")
    
    cfa_menu_data = []

    # Locate the nutrition table
    cfa_table = cfa_soup.find('table', class_='tblCompare')
    if not cfa_table:
        raise Exception("Chick-fil-A nutrition table not found")

    cfa_rows = cfa_table.find_all('tr', class_=['odd', 'even'])

    for row in cfa_rows:
        name_cell = row.find('td', class_='al')
        if not name_cell:
            continue

        name_link = name_cell.find('a', class_='nmItem')
        cfa_item_name = name_link.text.strip() if name_link else "Unknown"

        cfa_nutrition_cells = row.find_all('td', class_='col')
        if len(cfa_nutrition_cells) < 10:
            continue

        cfa_menu_data.append({
            'Item': cfa_item_name,
            'Calories': cfa_nutrition_cells[0].text.strip(),
            'Total Fat (g)': cfa_nutrition_cells[1].text.strip(),
            'Saturated Fat (g)': cfa_nutrition_cells[2].text.strip(),
            'Trans Fat (g)': cfa_nutrition_cells[3].text.strip(),
            'Cholesterol (mg)': cfa_nutrition_cells[4].text.strip(),
            'Sodium (mg)': cfa_nutrition_cells[5].text.strip().replace(',', ''),
            'Total Carbohydrates (g)': cfa_nutrition_cells[6].text.strip(),
            'Dietary Fiber (g)': cfa_nutrition_cells[7].text.strip(),
            'Sugars (g)': cfa_nutrition_cells[8].text.strip(),
            'Protein (g)': cfa_nutrition_cells[9].text.strip()
        })

    if not cfa_menu_data:
        raise Exception(f"No Chick-fil-A menu items found. Check the page source: {cfa_source_path}")

    cfa_df = pd.DataFrame(cfa_menu_data)
    print("\nFirst 5 Chick-fil-A items:")
    print(cfa_df.head())

    # Save the dataframe to your project directory
    cfa_csv_path = r"C:\Users\asmit\OneDrive\Desktop\fast_food_nutrition\data\chick_fil_a_nutrition.csv"
    cfa_df.to_csv(cfa_csv_path, index=False)
    print(f"\nSuccessfully saved {len(cfa_df)} items to: {cfa_csv_path}")

except Exception as e:
    print(f"\nError: {e}")
    cfa_screenshot_path = r"C:\Users\asmit\OneDrive\Desktop\fast_food_nutrition\data\cfa_error_screenshot.png"
    driver.save_screenshot(cfa_screenshot_path)
    print(f"Screenshot saved to: {cfa_screenshot_path}")
    print(f"Page source saved to: {cfa_source_path}")

finally:
    driver.quit()


Loading Chick-fil-A page...
Saved page source to: C:\Users\asmit\OneDrive\Desktop\fast_food_nutrition\data\cfa_page_source.html

First 5 Chick-fil-A items:
                               Item Calories Total Fat (g) Saturated Fat (g)  \
0       Bacon, Egg & Cheese Biscuit      420            23                11   
1        Bacon, Egg & Cheese Muffin      300            13                 6   
2  Berry Parfait with Cookie Crumbs      240             8               3.5   
3        Berry Parfait with Granola      270             9               3.5   
4                   Buttery Biscuit      290            15                 6   

  Trans Fat (g) Cholesterol (mg) Sodium (mg) Total Carbohydrates (g)  \
0             0              180        1220                      38   
1             0              180         780                      28   
2             0               20          85                      31   
3             0               20          85                      35   
4  

Scraping Burger King Menu

In [4]:
# Configure Chrome options
chrome_options = Options()
# chrome_options.add_argument("--headless")  # Uncomment when done testing
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--window-size=1920,1080")
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")

service = Service()
driver = webdriver.Chrome(service=service, options=chrome_options)

try:
    bk_url = "https://www.nutritionix.com/burger-king/menu/premium?desktop"
    driver.get(bk_url)
    
    time.sleep(3)  # Let JavaScript render

    # Parse the page source
    bk_soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Save HTML locally
    bk_source_path = r"C:\Users\asmit\OneDrive\Desktop\fast_food_nutrition\data\bk_page_source.html"
    with open(bk_source_path, 'w', encoding='utf-8') as f:
        f.write(bk_soup.prettify())
    print(f"Saved page source to: {bk_source_path}")
    
    bk_menu_data = []

    # Locate the nutrition table
    bk_table = bk_soup.find('table', class_='tblCompare')
    if not bk_table:
        raise Exception("Burger King nutrition table not found")

    bk_rows = bk_table.find_all('tr', class_=['odd', 'even'])

    for row in bk_rows:
        name_cell = row.find('td', class_='al')
        if not name_cell:
            continue

        name_link = name_cell.find('a', class_='nmItem')
        bk_item_name = name_link.text.strip() if name_link else "Unknown"

        bk_nutrition_cells = row.find_all('td', class_='col')
        if len(bk_nutrition_cells) < 10:
            continue

        bk_menu_data.append({
            'Item': bk_item_name,
            'Calories': bk_nutrition_cells[0].text.strip(),
            'Total Fat (g)': bk_nutrition_cells[1].text.strip(),
            'Saturated Fat (g)': bk_nutrition_cells[2].text.strip(),
            'Trans Fat (g)': bk_nutrition_cells[3].text.strip(),
            'Cholesterol (mg)': bk_nutrition_cells[4].text.strip(),
            'Sodium (mg)': bk_nutrition_cells[5].text.strip().replace(',', ''),
            'Total Carbohydrates (g)': bk_nutrition_cells[6].text.strip(),
            'Dietary Fiber (g)': bk_nutrition_cells[7].text.strip(),
            'Sugars (g)': bk_nutrition_cells[8].text.strip(),
            'Protein (g)': bk_nutrition_cells[9].text.strip()
        })

    if not bk_menu_data:
        raise Exception(f"No Burger King menu items found. Check the page source: {bk_source_path}")

    bk_df = pd.DataFrame(bk_menu_data)
    print("\nFirst 5 Burger King items:")
    print(bk_df.head())

    # Save the dataframe to your project directory
    bk_csv_path = r"C:\Users\asmit\OneDrive\Desktop\fast_food_nutrition\data\burger_king_nutrition.csv"
    bk_df.to_csv(bk_csv_path, index=False)
    print(f"\nSuccessfully saved {len(bk_df)} items to: {bk_csv_path}")

except Exception as e:
    print(f"\nError: {e}")
    bk_screenshot_path = r"C:\Users\asmit\OneDrive\Desktop\fast_food_nutrition\data\bk_error_screenshot.png"
    driver.save_screenshot(bk_screenshot_path)
    print(f"Screenshot saved to: {bk_screenshot_path}")
    print(f"Page source saved to: {bk_source_path}")

finally:
    driver.quit()


Loading Burger King page...
Saved page source to: C:\Users\asmit\OneDrive\Desktop\fast_food_nutrition\data\bk_page_source.html

First 5 Burger King items:
                                    Item Calories Total Fat (g)  \
0            3 Piece French Toast Sticks      350            12   
1            5 Piece French Toast Sticks      520            19   
2           Bacon, Egg, & Cheese Biscuit      450            30   
3     Bacon, Egg, & Cheese Croissan'wich      410            25   
4  Bacon, Sausage, Egg, & Cheese Biscuit      710            53   

  Saturated Fat (g) Trans Fat (g) Cholesterol (mg) Sodium (mg)  \
0                 2             0                0         220   
1                 4             0                0         350   
2                14             1              235        1580   
3                12             1              260        1030   
4                24             1              290        2390   

  Total Carbohydrates (g) Dietary Fiber (g) S

Scraping KFC menu

In [5]:
# Configure Chrome options
chrome_options = Options()
# chrome_options.add_argument("--headless")  # Uncomment for headless mode
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--window-size=1920,1080")
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")

service = Service()
driver = webdriver.Chrome(service=service, options=chrome_options)

try:
    kfc_url = "https://www.nutritionix.com/kfc/menu/premium?desktop"
    driver.get(kfc_url)
    
    time.sleep(3)  # Wait for JavaScript content to load

    kfc_soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Save HTML page source
    kfc_source_path = r"C:\Users\asmit\OneDrive\Desktop\fast_food_nutrition\data\kfc_page_source.html"
    with open(kfc_source_path, 'w', encoding='utf-8') as f:
        f.write(kfc_soup.prettify())
    print(f"Saved page source to: {kfc_source_path}")
    
    kfc_menu_data = []

    kfc_table = kfc_soup.find('table', class_='tblCompare')
    if not kfc_table:
        raise Exception("KFC nutrition table not found")

    kfc_rows = kfc_table.find_all('tr', class_=['odd', 'even'])

    for row in kfc_rows:
        name_cell = row.find('td', class_='al')
        if not name_cell:
            continue

        name_link = name_cell.find('a', class_='nmItem')
        kfc_item_name = name_link.text.strip() if name_link else "Unknown"

        kfc_nutrition_cells = row.find_all('td', class_='col')
        if len(kfc_nutrition_cells) < 10:
            continue

        kfc_menu_data.append({
            'Item': kfc_item_name,
            'Calories': kfc_nutrition_cells[0].text.strip(),
            'Total Fat (g)': kfc_nutrition_cells[1].text.strip(),
            'Saturated Fat (g)': kfc_nutrition_cells[2].text.strip(),
            'Trans Fat (g)': kfc_nutrition_cells[3].text.strip(),
            'Cholesterol (mg)': kfc_nutrition_cells[4].text.strip(),
            'Sodium (mg)': kfc_nutrition_cells[5].text.strip().replace(',', ''),
            'Total Carbohydrates (g)': kfc_nutrition_cells[6].text.strip(),
            'Dietary Fiber (g)': kfc_nutrition_cells[7].text.strip(),
            'Sugars (g)': kfc_nutrition_cells[8].text.strip(),
            'Protein (g)': kfc_nutrition_cells[9].text.strip()
        })

    if not kfc_menu_data:
        raise Exception(f"No KFC menu items found. Check the page source: {kfc_source_path}")

    kfc_df = pd.DataFrame(kfc_menu_data)
    print("\nFirst 5 KFC items:")
    print(kfc_df.head())

    # Save as CSV
    kfc_csv_path = r"C:\Users\asmit\OneDrive\Desktop\fast_food_nutrition\data\kfc_nutrition.csv"
    kfc_df.to_csv(kfc_csv_path, index=False)
    print(f"\nSuccessfully saved {len(kfc_df)} items to: {kfc_csv_path}")

except Exception as e:
    print(f"\nError: {e}")
    kfc_screenshot_path = r"C:\Users\asmit\OneDrive\Desktop\fast_food_nutrition\data\kfc_error_screenshot.png"
    driver.save_screenshot(kfc_screenshot_path)
    print(f"Screenshot saved to: {kfc_screenshot_path}")
    print(f"Page source saved to: {kfc_source_path}")

finally:
    driver.quit()


Loading KFC page...
Saved page source to: C:\Users\asmit\OneDrive\Desktop\fast_food_nutrition\data\kfc_page_source.html

First 5 KFC items:
                                    Item Calories Total Fat (g)  \
0                      Apple Pie Poppers       80             5   
1                     Cherry Pie Poppers       70           4.5   
2  KFC Chicken Sandwich - Chipotle Ranch      670            38   
3       KFC Chicken Sandwich - Honey BBQ      600            26   
4    KFC Chicken Sandwich - Honey Garlic      610            26   

  Saturated Fat (g) Trans Fat (g) Cholesterol (mg) Sodium (mg)  \
0               1.5             0                0          55   
1                 0             0                0          40   
2                 5             0               90        1330   
3                 3             0               80        1330   
4                 3             0               80        1370   

  Total Carbohydrates (g) Dietary Fiber (g) Sugars (g) Prote

Scraping SubWay menu

In [6]:
# Configure Chrome options
chrome_options = Options()
# chrome_options.add_argument("--headless")  # Enable this after testing
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--window-size=1920,1080")
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")

service = Service()
driver = webdriver.Chrome(service=service, options=chrome_options)

try:
    subway_url = "https://www.nutritionix.com/subway/menu/premium?desktop"
    driver.get(subway_url)
    
    time.sleep(3)  # Allow JavaScript to load
    
    subway_soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Save HTML page source
    subway_source_path = r"C:\Users\asmit\OneDrive\Desktop\fast_food_nutrition\data\subway_page_source.html"
    with open(subway_source_path, 'w', encoding='utf-8') as f:
        f.write(subway_soup.prettify())
    print(f"Saved page source to: {subway_source_path}")
    
    subway_menu_data = []

    subway_table = subway_soup.find('table', class_='tblCompare')
    if not subway_table:
        raise Exception("Subway nutrition table not found")

    subway_rows = subway_table.find_all('tr', class_=['odd', 'even'])

    for row in subway_rows:
        name_cell = row.find('td', class_='al')
        if not name_cell:
            continue

        name_link = name_cell.find('a', class_='nmItem')
        subway_item_name = name_link.text.strip() if name_link else "Unknown"

        subway_nutrition_cells = row.find_all('td', class_='col')
        if len(subway_nutrition_cells) < 10:
            continue

        subway_menu_data.append({
            'Item': subway_item_name,
            'Calories': subway_nutrition_cells[0].text.strip(),
            'Total Fat (g)': subway_nutrition_cells[1].text.strip(),
            'Saturated Fat (g)': subway_nutrition_cells[2].text.strip(),
            'Trans Fat (g)': subway_nutrition_cells[3].text.strip(),
            'Cholesterol (mg)': subway_nutrition_cells[4].text.strip(),
            'Sodium (mg)': subway_nutrition_cells[5].text.strip().replace(',', ''),
            'Total Carbohydrates (g)': subway_nutrition_cells[6].text.strip(),
            'Dietary Fiber (g)': subway_nutrition_cells[7].text.strip(),
            'Sugars (g)': subway_nutrition_cells[8].text.strip(),
            'Protein (g)': subway_nutrition_cells[9].text.strip()
        })

    if not subway_menu_data:
        raise Exception(f"No Subway menu items found. Check the page source: {subway_source_path}")

    subway_df = pd.DataFrame(subway_menu_data)
    print("\nFirst 5 Subway items:")
    print(subway_df.head())

    # Save to CSV
    subway_csv_path = r"C:\Users\asmit\OneDrive\Desktop\fast_food_nutrition\data\subway_nutrition.csv"
    subway_df.to_csv(subway_csv_path, index=False)
    print(f"\nSuccessfully saved {len(subway_df)} items to: {subway_csv_path}")

except Exception as e:
    print(f"\nError: {e}")
    subway_screenshot_path = r"C:\Users\asmit\OneDrive\Desktop\fast_food_nutrition\data\subway_error_screenshot.png"
    driver.save_screenshot(subway_screenshot_path)
    print(f"Screenshot saved to: {subway_screenshot_path}")
    print(f"Page source saved to: {subway_source_path}")

finally:
    driver.quit()


Loading Subway page...
Saved page source to: C:\Users\asmit\OneDrive\Desktop\fast_food_nutrition\data\subway_page_source.html

First 5 Subway items:
                       Item Calories Total Fat (g) Saturated Fat (g)  \
0          6" #1 The Philly      510             0                25   
1          6" #2 The Outlaw      490             0                22   
2            6" #6 The Boss      690             0                38   
3  6" #10 All-American Club      540             0                28   
4        6" #11 Subway Club      500             0                24   

  Trans Fat (g) Cholesterol (mg) Sodium (mg) Total Carbohydrates (g)  \
0             9                1          85                   1,320   
1             9                1          90                   1,440   
2            16                1          85                   1,860   
3            10                1          75                   1,520   
4             8                1          75              

Categorising menu items

In [9]:
# Path to data folder
data_dir = r"C:\Users\asmit\OneDrive\Desktop\fast_food_nutrition\data"

# List of CSV files and their corresponding chain names
chain_files = {
    "chick_fil_a_nutrition.csv": "Chick-fil-A",
    "burger_king_nutrition.csv": "Burger King",
    "kfc_nutrition.csv": "KFC",
    "subway_nutrition.csv": "Subway",
    "mcdonalds_nutrition.csv": "McDonald's"  # Assuming this was converted from PDF earlier
}

# Function to categorize items based on their name
def categorize_item(item):
    item = item.lower()
    if 'chicken' in item:
        return 'Chicken'
    elif 'burger' in item:
        return 'Burger'
    elif 'fish' in item:
        return 'Fish'
    elif 'salad' in item:
        return 'Salad'
    elif 'wrap' in item:
        return 'Wrap'
    elif 'sub' in item or 'sandwich' in item:
        return 'Sandwich'
    elif 'side' in item or 'fries' in item:
        return 'Side'
    elif 'drink' in item or 'beverage' in item:
        return 'Drink'
    else:
        return 'Other'

# Combine data
combined_df = pd.DataFrame()

for file_name, chain_name in chain_files.items():
    path = os.path.join(data_dir, file_name)
    if os.path.exists(path):
        print(f"Loading: {file_name}")
        df = pd.read_csv(path)

        # Add chain name
        df["Chain"] = chain_name

        # Ensure 'Item' column exists
        if "Item" in df.columns:
            df["category"] = df["Item"].apply(categorize_item)
        else:
            print(f"Warning: 'Item' column not found in {file_name}")
            df["category"] = "Unknown"

        combined_df = pd.concat([combined_df, df], ignore_index=True)
    else:
        print(f"File not found: {path}")

# Save master CSV
output_path = os.path.join(data_dir, "master_nutrition_data.csv")
combined_df.to_csv(output_path, index=False)
print(f"\nMaster dataset saved to: {output_path}")


Loading: chick_fil_a_nutrition.csv
Loading: burger_king_nutrition.csv
Loading: kfc_nutrition.csv
Loading: subway_nutrition.csv
Loading: mcdonalds_nutrition.csv

✅ Master dataset saved to: C:\Users\asmit\OneDrive\Desktop\fast_food_nutrition\data\master_nutrition_data.csv


Checking what the merged dataset looks like

In [11]:
# Load master dataset
merged_df = pd.read_csv(r"C:\Users\asmit\OneDrive\Desktop\fast_food_nutrition\data\master_nutrition_data.csv")

# Clean up column names: remove extra spaces
merged_df.columns = merged_df.columns.str.strip()

# Convert relevant columns to numeric
numeric_columns = [
    'Calories', 'Total Fat (g)', 'Saturated Fat (g)', 'Trans Fat (g)',
    'Cholesterol (mg)', 'Sodium (mg)', 'Total Carbohydrates (g)',
    'Dietary Fiber (g)', 'Sugars (g)', 'Protein (g)'
]

for col in numeric_columns:
    merged_df[col] = pd.to_numeric(merged_df[col], errors='coerce')  # convert and handle non-numeric values gracefully

# Check what the data looks like now
print(merged_df.head())
print("\nNull values per column:")
print(merged_df[numeric_columns].isnull().sum())


                               Item  Calories  Total Fat (g)  \
0       Bacon, Egg & Cheese Biscuit     420.0           23.0   
1        Bacon, Egg & Cheese Muffin     300.0           13.0   
2  Berry Parfait with Cookie Crumbs     240.0            8.0   
3        Berry Parfait with Granola     270.0            9.0   
4                   Buttery Biscuit     290.0           15.0   

   Saturated Fat (g)  Trans Fat (g)  Cholesterol (mg)  Sodium (mg)  \
0               11.0            0.0             180.0       1220.0   
1                6.0            0.0             180.0        780.0   
2                3.5            0.0              20.0         85.0   
3                3.5            0.0              20.0         85.0   
4                6.0            0.0               0.0        760.0   

   Total Carbohydrates (g)  Dietary Fiber (g)  Sugars (g)  Protein (g)  \
0                     38.0                2.0         4.0           15   
1                     28.0                1.0 

Dropping rows missing key nutrients for analysis aswell as making a protein per 100 calories column

In [15]:
# Load your dataset
merged_df = pd.read_csv(r"C:\Users\asmit\OneDrive\Desktop\fast_food_nutrition\data\master_nutrition_data.csv")

# Strip column names of leading/trailing whitespace
merged_df.columns = merged_df.columns.str.strip()

# Convert relevant columns to numeric
nutrient_cols = [
    'Calories', 'Total Fat (g)', 'Saturated Fat (g)', 'Trans Fat (g)',
    'Cholesterol (mg)', 'Sodium (mg)', 'Total Carbohydrates (g)',
    'Dietary Fiber (g)', 'Sugars (g)', 'Protein (g)'
]

# Ensure numeric types and handle non-numeric entries
for col in nutrient_cols:
    merged_df[col] = pd.to_numeric(merged_df[col], errors='coerce')

# Drop rows with missing key nutrients
merged_df.dropna(subset=['Calories', 'Protein (g)'], inplace=True)

# Add a new metric: Protein per 100 Calories
merged_df['Protein per 100 Cal'] = (merged_df['Protein (g)'] / merged_df['Calories']) * 100

# Optional: Clean up restaurant names
merged_df['Chain'] = merged_df['Chain'].str.strip().str.title()

# Preview
merged_df.head()


,Item,Calories,Total Fat (g),Saturated Fat (g),Trans Fat (g),Cholesterol (mg),Sodium (mg),Total Carbohydrates (g),Dietary Fiber (g),Sugars (g),Protein (g),Chain,category,Protein per 100 Cal
0,"Bacon, Egg & Cheese Biscuit",420.0,23.0,11.0,0.0,180.0,1220.0,38.0,2.0,4.0,15,Chick-Fil-A,Other,3.571429
1,"Bacon, Egg & Cheese Muffin",300.0,13.0,6.0,0.0,180.0,780.0,28.0,1.0,2.0,16,Chick-Fil-A,Other,5.333333
2,Berry Parfait with Cookie Crumbs,240.0,8.0,3.5,0.0,20.0,85.0,31.0,1.0,25.0,12,Chick-Fil-A,Other,5.000000
3,Berry Parfait with Granola,270.0,9.0,3.5,0.0,20.0,85.0,35.0,1.0,26.0,13,Chick-Fil-A,Other,4.814815
4,Buttery Biscuit,290.0,15.0,6.0,0.0,0.0,760.0,37.0,2.0,4.0,4,Chick-Fil-A,Other,1.379310


Further data cleaning

In [16]:
# Load master dataset
merged_df = pd.read_csv(r"C:\Users\asmit\OneDrive\Desktop\fast_food_nutrition\data\master_nutrition_data.csv")

# Clean the 'Chain' column to ensure uniformity
merged_df['Chain'] = merged_df['Chain'].str.strip().str.title()

# Save the cleaned dataset to a new CSV file
output_path = r"C:\Users\asmit\OneDrive\Desktop\fast_food_nutrition\data\cleaned_nutrition_data.csv"
merged_df.to_csv(output_path, index=False)

print(f"\nCleaned data saved to: {output_path}")



✅ Cleaned data saved to: C:\Users\asmit\OneDrive\Desktop\fast_food_nutrition\data\cleaned_nutrition_data.csv


Finding most common words to categorise

In [31]:
from collections import Counter
import re

# Load your dataset
merged_df = pd.read_csv(r"C:\Users\asmit\OneDrive\Desktop\fast_food_nutrition\data\master_nutrition_data.csv")

# Clean column names just in case
merged_df.columns = merged_df.columns.str.strip()

# Combine all item names into one string, lowercase, and split into words
all_items = " ".join(merged_df['Item'].dropna().astype(str)).lower()

# Remove punctuation and split into words
words = re.findall(r'\b\w+\b', all_items)

# Remove common stop words
stopwords = {'with', 'and', 'or', 'a', 'the', 'of', 'on', 'to', 'for', 'in', 'at', 'from'}
filtered_words = [word for word in words if word not in stopwords]

# Count frequency
word_counts = Counter(filtered_words)

# Show the 30 most common words
most_common = word_counts.most_common(30)
for word, count in most_common:
    print(f"{word}: {count}")


chicken: 122
small: 107
large: 98
fl: 98
oz: 98
medium: 90
chick: 77
cheese: 76
1: 69
tea: 68
salad: 64
lemonade: 64
iced: 63
fil: 61
6: 57
grilled: 52
2: 52
spicy: 48
wrap: 48
bowl: 46
hash: 44
diet: 44
tray: 43
bacon: 42
nuggets: 40
egg: 39
chocolate: 34
sweet: 33
12: 31
sauce: 31


Creating categories for different types of food to aid analysis

In [27]:
# Drop old 'category' column if it exists
if 'category' in merged_df.columns:
    merged_df.drop(columns=['category'], inplace=True)

# Define category keywords
category_keywords = {
    'Chicken': ['chicken', 'chick', 'nuggets', 'fil', 'grilled', 'spicy'],
    'Salad': ['salad'],
    'Wrap': ['wrap'],
    'Burger': ['burger'],
    'Beverage': ['tea', 'lemonade', 'iced', 'diet', 'sweet', 'drink', 'juice', 'soda'],
    'Dessert': ['chocolate', 'cookie', 'ice cream', 'dessert'],
    'Breakfast': ['egg', 'hash', 'bacon'],
    'Side': ['sauce', 'tray', 'side', 'fries'],
    'Combo': ['bowl', '1', '2', '6', '12']  # optional utility grouping
}

# Function to assign categories
def categorize_item_multi(name):
    name = name.lower()
    matched = set()
    for category, keywords in category_keywords.items():
        if any(keyword in name for keyword in keywords):
            matched.add(category)
    if not matched:
        return ['Other']
    return list(matched)

# Apply multi-label categorization
merged_df['Category'] = merged_df['Item'].apply(categorize_item_multi)

# Preview results
merged_df


,Item,Calories,Total Fat (g),Saturated Fat (g),Trans Fat (g),Cholesterol (mg),Sodium (mg),Total Carbohydrates (g),Dietary Fiber (g),Sugars (g),Protein (g),Chain,Category
0,"Bacon, Egg & Cheese Biscuit",420,23.0,11.0,0.0,180,1220,38,2,4,15,Chick-fil-A,[Breakfast]
1,"Bacon, Egg & Cheese Muffin",300,13.0,6.0,0.0,180,780,28,1,2,16,Chick-fil-A,[Breakfast]
2,Berry Parfait with Cookie Crumbs,240,8.0,3.5,0.0,20,85,31,1,25,12,Chick-fil-A,[Dessert]
3,Berry Parfait with Granola,270,9.0,3.5,0.0,20,85,35,1,26,13,Chick-fil-A,[Other]
4,Buttery Biscuit,290,15.0,6.0,0.0,0,760,37,2,4,4,Chick-fil-A,[Other]
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1109,"Sweet Tea, Small",100,0.0,0.0,0.0,0,15,26,0,24,24,McDonald's,[Beverage]
1110,"Unsweetened Iced Tea, Extra Small",0,0.0,0.0,0.0,0,10,0,0,0,0,McDonald's,[Beverage]
1111,"Unsweetened Iced Tea, Large",0,0.0,0.0,0.0,0,20,0,0,0,0,McDonald's,[Beverage]
1112,"Unsweetened Iced Tea, Medium",0,0.0,0.0,0.0,0,15,0,0,0,0,McDonald's,[Beverage]


Categorising into small medium and large

In [29]:
# Define allowed size labels
valid_sizes = ['small', 'medium', 'large']

# Function to extract size
def extract_size(name):
    name = name.lower()
    for size in valid_sizes:
        if size in name:
            return size.capitalize()
    return 'N/A'

# Apply function to create Size column
merged_df['Size'] = merged_df['Item'].apply(extract_size)

# Preview
merged_df


,Item,Calories,Total Fat (g),Saturated Fat (g),Trans Fat (g),Cholesterol (mg),Sodium (mg),Total Carbohydrates (g),Dietary Fiber (g),Sugars (g),Protein (g),Chain,Category,Size
0,"Bacon, Egg & Cheese Biscuit",420,23.0,11.0,0.0,180,1220,38,2,4,15,Chick-fil-A,[Breakfast],N/A
1,"Bacon, Egg & Cheese Muffin",300,13.0,6.0,0.0,180,780,28,1,2,16,Chick-fil-A,[Breakfast],N/A
2,Berry Parfait with Cookie Crumbs,240,8.0,3.5,0.0,20,85,31,1,25,12,Chick-fil-A,[Dessert],N/A
3,Berry Parfait with Granola,270,9.0,3.5,0.0,20,85,35,1,26,13,Chick-fil-A,[Other],N/A
4,Buttery Biscuit,290,15.0,6.0,0.0,0,760,37,2,4,4,Chick-fil-A,[Other],N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1109,"Sweet Tea, Small",100,0.0,0.0,0.0,0,15,26,0,24,24,McDonald's,[Beverage],Small
1110,"Unsweetened Iced Tea, Extra Small",0,0.0,0.0,0.0,0,10,0,0,0,0,McDonald's,[Beverage],Small
1111,"Unsweetened Iced Tea, Large",0,0.0,0.0,0.0,0,20,0,0,0,0,McDonald's,[Beverage],Large
1112,"Unsweetened Iced Tea, Medium",0,0.0,0.0,0.0,0,15,0,0,0,0,McDonald's,[Beverage],Medium
